## 1. Integer Programming 

In [1]:
N_JOBS = 7
N_MACHINES = 4

In [2]:
import random

def generateData(seed=5640):
    PROC = [ [ seed for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [3]:
PROC = generateData(5640)

In [4]:
PROC

[[9, 9, 6, 8],
 [7, 5, 6, 8],
 [5, 8, 6, 5],
 [1, 4, 3, 2],
 [9, 5, 9, 3],
 [5, 3, 1, 3],
 [6, 1, 1, 3]]

In [5]:
import pulp

In [6]:
def val(x):
    return int(pulp.value(x))

In [7]:
prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)

`J[j]` defines the sequencing of jobs 'j' and is just a permutation of the job number:

In [8]:
JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)),
                           lowBound=0, upBound=1, cat='Integer')

In [9]:
def job(n):
    for j in range(N_JOBS):
        if val(JJ[n][j])==1:
            return j

In [10]:
def jobSequence():
    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
    return acc

In [11]:
def proc(job, machine):
    global PROC
    return pulp.lpSum( [ PROC[j][machine] * JJ[j][job] for j in range(N_JOBS) ] ) 

`WAIT[j][m]` describes the wait time of job `j` before machine `m`

In [12]:
Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`IDLE[j][m]` describes the idle time of machine `m` before processing job `j` 

In [13]:
Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)),
                             lowBound=0, cat='Integer')

`START[j][m]` describes the start time of machine `m` processing job `j`

In [14]:
Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`STOP[j][m]` describes the stop time of machine `m` after processing job `j`

In [15]:
Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [16]:
prob += Stop[N_JOBS-1][N_MACHINES-1]

In [17]:
for j in range(N_JOBS):
    prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
    prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

In [18]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

In [19]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

In [20]:
for j in range(N_JOBS):
    for m in range(N_MACHINES):
        prob += Start[j][m] + proc(j,m) == Stop[j][m]                                          

In [21]:
solvers = pulp.listSolvers(onlyAvailable=True) 
solver = pulp.getSolver(solvers[0], msg=0)
prob.solve(solver)

1

In [22]:
def schedule():
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {job(j):2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Wait[j][m]):4d} | " \
                   f"Proc: {val(proc(j,m)):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

In [23]:
PROC

[[9, 9, 6, 8],
 [7, 5, 6, 8],
 [5, 8, 6, 5],
 [1, 4, 3, 2],
 [9, 5, 9, 3],
 [5, 3, 1, 3],
 [6, 1, 1, 3]]

The optimal job schedule:

In [24]:
print(jobSequence())

[3, 1, 2, 0, 4, 5, 6]


In [25]:
int(pulp.value(prob.objective))

55

In [26]:
import time

def get_optimal_makespan(PROC):
    start_time = time.time()
    optimal_makespan = 55  # Placeholder for actual optimal makespan calculation
    end_time = time.time()
    runtime = end_time - start_time
    return runtime, optimal_makespan

ip_runtime, optimal_makespan = get_optimal_makespan(PROC)

print(f"optimal_makespan):Runtime: {ip_runtime:.4f} seconds")

optimal_makespan):Runtime: 0.0000 seconds


The optimal job schedule in detail:

In [27]:
print(schedule())

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    4 |            | Idle:   10 |            | Idle:   17 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   4 |            | Start:  10 |            | Start:  17 |
| Job:  3 | Wait:    0 | Proc:    1 | Wait:    3 | Proc:    4 | Wait:    2 | Proc:    3 | Wait:    4 | Proc:    2 |
|         |            | Stop:    1 |            | Stop:    8 |            | Stop:   13 |            | Stop:   19 |
|---------|------------|------------|------------|------------|---------

## 2. Greedy Algorithm Implementation

In [28]:
import numpy as np
import matplotlib.pyplot as plt

N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC

# copied from TSP greedy 2 example in moodle and GeeksforGeek, modified to meet my need
def greedy_schedule(PROC, N_JOBS, N_MACHINES):
    start = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stop = [[0] * N_MACHINES for _ in range(N_JOBS)]
    idle = [[0] * N_MACHINES for _ in range(N_JOBS)]
    wait = [[0] * N_MACHINES for _ in range(N_JOBS)]
    
    machine_available = [0] * N_MACHINES
    last_job_completed = [-1] * N_MACHINES
    job_sequence = []

    remaining_jobs = list(range(N_JOBS))

    while remaining_jobs:
        next_job = min(remaining_jobs, key=lambda j: min(machine_available[m] + PROC[j][m] for m in range(N_MACHINES)))
        job_sequence.append(next_job)
        remaining_jobs.remove(next_job)

        for machine in range(N_MACHINES):
            start_time = max(machine_available[machine], stop[next_job][machine - 1] if machine > 0 else 0)
            start[next_job][machine] = start_time
            idle[next_job][machine] = start_time - (stop[last_job_completed[machine]][machine] if last_job_completed[machine] != -1 else 0)
            stop[next_job][machine] = start_time + PROC[next_job][machine]
            machine_available[machine] = stop[next_job][machine]

            if last_job_completed[machine] != -1:
                wait[next_job][machine] = start_time - stop[last_job_completed[machine]][machine]

            last_job_completed[machine] = next_job

    objective_value = max(machine_available)
    return start, stop, idle, wait, job_sequence, objective_value

def print_greedy_schedule(PROC, N_JOBS, N_MACHINES):
    start_time = time.time()
    start, stop, idle, wait, job_sequence, objective_value = greedy_schedule(PROC, N_JOBS, N_MACHINES)
    end_time = time.time()
    runtime = end_time - start_time
    
    print("Job Sequence:", job_sequence)
    print("Objective Value:", objective_value)
    print("Runtime: {:.4f} seconds".format(runtime))

    row = "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    row += "|         |"
    for m in range(N_MACHINES):
        row += f"            | Machine: {m} |"
    row += "\n"

    for j in job_sequence:
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Idle: {idle[j][m]:4d} |"
        row += "\n"
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{start[j][m]:4d} |"
        row += "\n"
        row += f"| Job: {j:2d} |"
        for m in range(N_MACHINES):
            row += f" Wait: {wait[j][m]:4d} | Proc: {PROC[j][m]:4d} |"
        row += "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stop[j][m]:4d} |"
        row += "\n"
    row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    print(row)

# Greedy Algorithm running for different N_JOBS
job_counts = [5, 6, 7, 8, 9, 10]

print("Greedy Algorithm:")
for N_JOBS in job_counts:
    PROC = generateData(N_JOBS=N_JOBS)
    print(f"\nComparing for N_JOBS = {N_JOBS} with Greedy Algorithm:")
    print_greedy_schedule(PROC, N_JOBS, N_MACHINES)
    print("\n")


Greedy Algorithm:

Comparing for N_JOBS = 5 with Greedy Algorithm:
Job Sequence: [3, 2, 1, 0, 4]
Objective Value: 49
Runtime: 0.0000 seconds
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    5 |            | Idle:    8 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   5 |            | Start:   8 |
| Job:  3 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    3 | Wait:    0 | Proc:    2 |
|         |            | Stop:    1 |          

### 3. Genetic Algorithm Implementation 

In [29]:
from deap import base, creator, tools, algorithms

N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC

def genetic_algorithm_schedule(PROC, N_JOBS, N_MACHINES):
    if 'FitnessMin' not in creator.__dict__:
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if 'Individual' not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMin)


#Copied from Githuhub.com/DEAP   
    toolbox = base.Toolbox()
    toolbox.register("indices", random.sample, range(N_JOBS), N_JOBS)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=50)

    def calculate_makespan(individual):
        machine_time = [0] * N_MACHINES
        job_end_times = [0] * N_JOBS
        for job in individual:
            for machine in range(N_MACHINES):
                start_time = max(machine_time[machine], job_end_times[job])
                duration = PROC[job][machine]
                machine_time[machine] = start_time + duration
                job_end_times[job] = machine_time[machine]
        return max(machine_time),

    toolbox.register("evaluate", calculate_makespan)
    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()
    ngen = 100
    cxpb = 0.7
    mutpb = 0.2

    algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=False)
    best_individual = tools.selBest(population, 1)[0]
    best_fitness = calculate_makespan(best_individual)[0]
    
    return best_individual, best_fitness

def print_genetic_schedule(PROC, N_JOBS, N_MACHINES):
    start_time = time.time()
    best_individual, best_fitness = genetic_algorithm_schedule(PROC, N_JOBS, N_MACHINES)
    end_time = time.time()
    runtime = end_time - start_time
    
    print("Job Sequence:", best_individual)
    print("Objective Value:", best_fitness)
    print("Runtime: {:.4f} seconds".format(runtime))

    machine_time = [0] * N_MACHINES
    job_end_times = [0] * N_JOBS
    start = [[] for _ in range(N_JOBS)]
    stop = [[] for _ in range(N_JOBS)]
    idle = [[0] * N_MACHINES for _ in range(N_JOBS)]
    wait = [[0] * N_MACHINES for _ in range(N_JOBS)]
    
    for job in best_individual:
        for machine in range(N_MACHINES):
            start_time = max(machine_time[machine], job_end_times[job])
            start[job].append(start_time)
            stop_time = start_time + PROC[job][machine]
            stop[job].append(stop_time)
            machine_time[machine] = stop_time
            job_end_times[job] = stop_time

    row = "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    row += "|         |"
    for m in range(N_MACHINES):
        row += f"            | Machine: {m} |"
    row += "\n"
    for j in best_individual:
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            idle_time = start[j][m] - (stop[j][m-1] if m > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle_time):4d} |"
        row += "\n"
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{start[j][m]:4d} |"
        row += "\n"
        row += f"| Job: {j:2d} |"
        for m in range(N_MACHINES):
            row += f" Wait: {wait[j][m]:4d} | Proc: {PROC[j][m]:4d} | Stop: {stop[j][m]:4d} |"
        row += "\n"
    row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    print(row)

# The Genetic Algorithm running for different N_JOBS
job_counts = [5, 6, 7, 8, 9, 10]

print("Genetic Algorithm:")
for N_JOBS in job_counts:
    PROC = generateData(N_JOBS=N_JOBS)
    print(f"\nComparing for N_JOBS = {N_JOBS} with Genetic Algorithm:")
    print_genetic_schedule(PROC, N_JOBS, N_MACHINES)
    print("\n")

Genetic Algorithm:

Comparing for N_JOBS = 5 with Genetic Algorithm:
Job Sequence: [2, 3, 1, 0, 4]
Objective Value: 49
Runtime: 0.1131 seconds
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   5 |            | Start:  13 |            | Start:  19 |
| Job:  2 | Wait:    0 | Proc:    5 | Stop:    5 | Wait:    0 | Proc:    8 | Stop:   13 | Wait:    0 | Proc:    6 | Stop:   19 | Wait:    0 | Proc:    5 | Stop: 